<a href="https://colab.research.google.com/github/AITrading1995/AITrading1995/blob/main/Multi_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git mplfinance backtesting pandas_ta numpy==1.23.5

In [ ]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import mplfinance as mpf
import math
import numpy as np
from backtesting import Backtest, Strategy
from tvDatafeed import TvDatafeed, Interval
import datetime as dt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def download(symbol,exchange,tf,bar):
  tf_map = {
      '1m': Interval.in_1_minute,
      '5m': Interval.in_5_minute,
      '15m': Interval.in_15_minute,
      '30m': Interval.in_30_minute,
      '1h': Interval.in_1_hour,
      '2h': Interval.in_2_hour,
      '4h': Interval.in_4_hour,
      '1d': Interval.in_daily
  }
  tv = TvDatafeed()
  data = tv.get_hist(symbol=symbol,exchange=exchange,interval=tf_map[tf],n_bars=bar)
  if data is not None:
    data.drop(columns='symbol', inplace=True)
    data.rename({
        'open':'Open'
        ,'high':'High'
        ,'low':'Low'
        ,'close':'Close'
        ,'volume':'Volume'}, axis='columns', inplace=True)
  return data

In [ ]:
symbol = 'XAUUSD'
exchange = 'OANDA'
tf = '1h'
bar = 10000
df = download(symbol,exchange,tf,bar)

In [ ]:
multi_symbol = ['XAUUSD','USDJPY','AUDNZD','GBPEUR','USDCAD','AUDCAD','AUDJPY','CADJPY','USDTHB','XAGUSD','US30','CAC40']
multi_data= {}
for sym in multi_symbol:
  data=download(sym,'OANDA','1h',10000)
  multi_data[sym] = data

In [ ]:
multi_data_list =[]
for sym in multi_symbol:
  data=download(sym,'OANDA','1h',10000)
  multi_data_list.append(data)

In [ ]:
multi_data.keys()

In [ ]:
dfs = {symbol : df['Close'] for symbol , df in multi_data.items() if df is not None}
dfs = pd.DataFrame(dfs)

In [ ]:
dfs.dropna(inplace=True)

In [ ]:
returns = np.log(dfs / dfs.shift(1))
corr = returns.corr()

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(corr, annot=True, cmap='coolwarm',linewidths=1)
plt.show()

In [ ]:
def parkinson_vol(high,low,length):
  high = np.asanyarray(high)
  low = np.asanyarray(low)
  n = high.shape[0]
  vol = np.full(n,np.nan)

  log_high_sqrt = np.log(high/low)**2
  for i in range(length,n):
    sum_sqrt = np.sum(log_high_sqrt[i-length:i])
    vol[i] = np.sqrt((1/(4*length*np.log(2)))*(2*sum_sqrt))
  return vol

In [ ]:
def ema(data, period):
    if data is None or len(data) == 0:
        return np.array([])
    return pd.Series(data).ewm(span=period, adjust=False).mean().to_numpy()

def sma(data, period):
    if data is None or len(data) == 0:
        return np.array([])
    return pd.Series(data).rolling(window=period).mean().to_numpy()

def derivertive_1(df):
  return np.gradient(df)
def derivertive_2(df):
  return np.gradient(derivertive_1(df))

In [ ]:
df['vol'] = parkinson_vol(df['High'],df['Low'],30)
df['ema_vol'] = sma(df['vol'],30)
df['ema'] = sma(df['Close'],50)
df['derivertive_1'] = derivertive_1(df['vol'])
df['derivertive_2'] = derivertive_2(df['vol'])

print(df.index[-1])

#Logic
Close > ema
derivertive_1 > 0.000004
vol.shift(1) > vol_ema



In [ ]:
start = '2025-05-01 '
end = '2025-05-25'

dfp = df.loc[start:end]

# Condition: vol shift >= ema_vol
markers_data = pd.Series(np.nan, index=dfp.index)
condition_marker = dfp['vol'].shift(1) >= dfp['ema_vol']
markers_data[condition_marker] = dfp['ema'][condition_marker]

# Derivative condition
derivertive_signal_series = pd.Series(np.nan, index=dfp.index)
condition_derivative = dfp['derivertive_1'] > 1e-8
derivertive_signal_series[condition_derivative] = dfp['Close'][condition_derivative]

# Buy condition
buy_sig = pd.Series(np.nan, index=dfp.index)
buy_con = (
    (dfp['Close'] > dfp['ema']) &
    (dfp['derivertive_1'] > 1e-8) &
    (dfp['vol'].shift(1) > dfp['ema_vol'])
)
buy_sig[buy_con] = dfp['Close'][buy_con]

# 🔻 Sell condition
sell_sig = pd.Series(np.nan, index=dfp.index)
sell_con = (
    (dfp['Close'] < dfp['ema']) &
    (dfp['derivertive_1'] > 1e-8) &
    (dfp['vol'].shift(1) > dfp['ema_vol'])
)
sell_sig[sell_con] = dfp['Close'][sell_con]

# Additional plots
add_plot = [
    mpf.make_addplot(dfp['ema'], color='blue', width=1),
    mpf.make_addplot(markers_data, type='scatter', markersize=50, marker='o', color='g', alpha=0.7),
    #mpf.make_addplot(derivertive_signal_series, type='scatter', markersize=50, marker='o', color='blue', alpha=0.7),
    mpf.make_addplot(dfp['vol'], color='red', width=1, panel=1),
    mpf.make_addplot(dfp['ema_vol'], color='green', width=1, panel=1),
    mpf.make_addplot(buy_sig, type='scatter', markersize=80, marker='^', color='lime', label='Buy Signal'),
    mpf.make_addplot(sell_sig, type='scatter', markersize=80, marker='v', color='red', label='Sell Signal')
]

# Plotting
mpf.plot(
    dfp,
    type='candle',
    style='binance',
    figratio=(30, 9),
    figscale=1.5,
    addplot=add_plot
)

plt.show()

In [ ]:
price = df.Close.iloc[-1]
sl = 0.004
tp = 0.008

sl = price *sl
tp = price *tp
print('sl',sl)
print('tp',tp)
sl_ =price-sl
tp_ =price+tp
print('last',price)
print('real ls' ,sl_ )
print('real tp' ,tp_)

In [ ]:
class Momentum(Strategy):
    # Strategy parameters (class-level defaults)
    ema_param = 43
    vol_param = 94
    mu_vol = 30
    sl_pct = 0.010
    tp_pct = 0.012

    def init(self):
        # Use self.data.Close instead of df.Close
        self.close = self.data.Close

        self.ema = self.I(ema, self.close, self.ema_param)
        self.vol = self.I(parkinson_vol, self.data.High, self.data.Low, self.vol_param)
        self.derivertive_1 = self.I(derivertive_1, self.vol)
        self.sma_vol = self.I(sma, self.vol, self.mu_vol)

    def next(self):
        price = self.close[-1]
        sl_amount = price * self.sl_pct
        tp_amount = price * self.tp_pct

        # Get the latest values
        ema_now = self.ema[-1]
        d1_now = self.derivertive_1[-1]
        vol_prev = self.vol[-2]
        sma_vol_now = self.sma_vol[-1]

        buy_con = (
            (price > ema_now) and
            (d1_now > 1e-8) and
            (vol_prev > sma_vol_now)
        )

        sell_con = (
            (price < ema_now) and
            (d1_now > 1e-8) and
            (vol_prev > sma_vol_now)
        )


        if  self.position:
          pass
        elif buy_con:
                self.buy(limit=price,size=1,sl=price - sl_amount, tp=price + tp_amount)
        if self.position:
            pass
        elif sell_con:
             self.sell(limit=price,size=1,sl=price + sl_amount, tp=price- tp_amount)

In [ ]:
bt = Backtest(
    df,
    Momentum,
    cash=10000,
    commission=0.00002,
    exclusive_orders=False,
    trade_on_close=True
)
print(bt.run())

In [ ]:
bt.plot(plot_trades=True,plot_equity=True,plot_drawdown=True,plot_volume=False,plot_return=False,show_legend=True)

In [ ]:
bt = Backtest(
    df,
    Momentum,
    cash=10000,
    commission=0.00002,
    exclusive_orders=False,
    trade_on_close=True,
)

stats, heatmap = bt.optimize(
    ema_param=range(10, 100, 3),
    vol_param=range(10, 100, 3),
    mu_vol=range(10, 100, 5),
    sl_pct=[0.002, 0.004, 0.006, 0.008, 0.010],
    tp_pct=[0.004, 0.006, 0.008, 0.010, 0.012],
    constraint=lambda p: p.tp_pct > p.sl_pct,
    maximize='Equity Final [$]',
    max_tries=500,
    random_state=42,
    return_heatmap=True
)

In [ ]:
from backtesting.lib import  plot_heatmaps
plot_heatmaps(heatmap, agg='mean')


In [ ]:
print(stats)

In [ ]:
heatmap.sort_values().iloc[-5:]

In [ ]:
multi_data_list

In [ ]:
for symbol, data in multi_data.items():
    if data is None:
        print(f"Skipping {symbol} because data is None")
        continue

    bt = Backtest(
        data,
        Momentum,
        cash=10_000,
        commission=0.00002,
        exclusive_orders=True,
        trade_on_close=True,
    )
    stats = bt.run()
    print(f"Backtest results for symbol {symbol}:")
    print(stats)
    #bt.plot() กรณี ต้องการ plot ผลลัพทั้งหมด